In [ ]:
g

## Predicting the Number of  People Have Tried the Recipe Based on the Food Recipes Data

#### BA 865 

#### Team members: Zixing Li, Hanyu Chen, Mengxin Li, Yapei Xiong

In [ ]:
#installs
! pip install scikit-plot
! pip install textdescriptives
! pip install tokenwiser
! pip install spacytextblob

     |████████████████████████████████| 2.0 MB 8.4 MB/s 
     |████████████████████████████████| 6.0 MB 46.3 MB/s 
     |████████████████████████████████| 653 kB 55.4 MB/s 
     |████████████████████████████████| 10.1 MB 39.9 MB/s 
     |████████████████████████████████| 181 kB 61.4 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 451 kB 55.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled 

In [ ]:
# imports
from sklearn.datasets import load_digits, fetch_openml, make_blobs
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import uuid
import random

# sklearn does have some functionality too, but mostly a wrapper to scipy
from sklearn.metrics import pairwise_distances 
from sklearn.preprocessing import StandardScaler
import scikitplot as skplt
import seaborn as sns
import matplotlib.pyplot as plt

# text imports
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer  
import nltk


# text imports
from spacy import cli
import textdescriptives as td
import tokenwiser
from sklearn.pipeline import Pipeline
nltk.download('punkt')
import gensim

import re

from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Data cleaning 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Importing recipe data from local
#df=pd.read_csv("/content/drive/MyDrive/food_recipes_data.csv")
#df = pd.read_csv('/content/drive/MyDrive/875 supply chain/dataset/food_recipes_data.csv') #Penny
df = pd. read_csv("food_recipes_data.csv")

In [ ]:
df.shape

(14200, 18)

In [ ]:
df.isna().sum()

total_time         2011
cook_time          3859
serving            2076
crawled_at            0
description           0
title                 0
url                   0
nutritions_info    2197
image                 0
ingredients         137
uniq_id               0
source                0
author                0
prep_time          3090
published_date        0
keywords            360
total_ratings        53
instructions       1959
dtype: int64

In [ ]:
df.head(3)

,total_time,cook_time,serving,crawled_at,description,title,url,nutritions_info,image,ingredients,uniq_id,source,author,prep_time,published_date,keywords,total_ratings,instructions
0,PT10M,NaN,14,2021-12-17 08:46:25,Create a delicious homemade chocolate bark wit...,Easy chocolate bark,https://www.bbcgoodfood.com/recipes/easy-choco...,calories:116 calories | fatContent:7 grams fat...,https://images.immediate.co.uk/production/vola...,"200g dark chocolate , chopped|2 tbsp chocolate...",6619d1f2-7c2c-5ab0-85f8-b8a1c190915e,BBC Good Food,Good Food team,PT10M,2020-02-17T15:47:46+00:00,"bark, BBC Good Food team, Chocolate, Dessert, ...",5 ratings,Melt the chocolate in short bursts in the micr...
1,PT55M,PT30M,2,2021-12-17 01:49:12,Steak and chips gets even more special with ra...,Venison steaks with stroganoff sauce & shoestr...,https://www.bbcgoodfood.com/recipes/venison-st...,calories:719 calories | fatContent:33 grams fa...,https://images.immediate.co.uk/production/vola...,"sunflower oil , for frying|2 large potatoes ,...",dc28ef65-01aa-527a-972d-0f959319e6ae,BBC Good Food,James Martin,PT25M,2012-10-09T04:20:39+00:00,"2 of 5-a-day, Autumnal, Deer, Fibre, Five a da...",24 ratings,"To make the sauce, heat 1 tbsp of the butter i..."
2,PT30M,PT20M,4,2021-12-17 02:50:12,A dish with added feelgood factor - this vitam...,Mexican chicken & wild rice soup,https://www.bbcgoodfood.com/recipes/mexican-ch...,calories:347 calories | fatContent:7 grams fat...,https://images.immediate.co.uk/production/vola...,"1 tsp olive oil|1 onion , finely chopped|1 gre...",f27090fa-5eaa-5472-8748-4954aa1ec802,BBC Good Food,Good Food team,PT10M,2011-06-14T01:36:59+00:00,"Josh Eagleton, 10-30 minutes, 30-60 minutes, ...",13 ratings,Heat the oil in a large non-stick frying pan a...


In [ ]:
def time_to_minutes(dtime):
    #print(dtime.head())
    data=dtime.str.findall(r'\d+').to_list()
    minutes = []
    for i in range(len(data)):
        data_i = list(map(int, data[i]))
        if len(data_i)==1:
            if re.match(r'^PT', dtime.iloc[i]):
                minutes.append(data_i[0])
            else:
                #print('1:',dtime.iloc[i],data_i)
                minutes.append(data_i[0]*24*60)  

        elif len(data_i)==2:
            if re.match(r'^PT',dtime.iloc[i]):
                minutes.append(data_i[0]*60+data_i[1])
            else: 
                #print('2:',dtime.iloc[i],data_i)
                minutes.append(data_i[0]*24*60+data_i[1]*60)
        else:
            #print('3:',dtime.iloc[i], data_i)
            minutes.append(data_i[0]*24*60+data_i[1]*60+data_i[2])
            
    return minutes

In [ ]:
def date_to_timestamp(ddate):
    ddate = pd.to_datetime(ddate).values.astype(np.int64)
    return ddate

In [ ]:
def serving_values(dd,debug_flag=0):
    #print(dtime.head())
    data=dd.str.findall(r'\d+').to_list()
    serving = []
    for i in range(len(data)):
        data_i = list(map(int, data[i]))
        if (len(data_i)==0 or len(data_i)>3):
            if debug_flag:
                print('0 or >3:', dd.iloc[i], data_i)
            serving.append(1)
        elif len(data_i)==1:
            serving.append(data_i[0])
        elif len(data_i)==2:
            if debug_flag:
                print('2:',dd.iloc[i],data_i)
            serving.append(sum(data_i)/len(data_i))
        else:
            if debug_flag:
                print('3:',dd.iloc[i], data_i)
            serving.append(data_i[0])
            
    return serving

In [ ]:
def extract_values_from_column(data):
    data1 = data.tolist()
    for i in range(len(data1)):
        if data1[i] is None:
            data1[i] = 0
        else:
            data1[i] = float(data1[i][0])
    
    return data1

In [ ]:
from sklearn import preprocessing
def author_encode(data):
    le = preprocessing.LabelEncoder()
    le.fit(data)
    data = le.fit_transform(data)
    return data, le

In [ ]:
def keywords_vectorizer(data):
    vect = CountVectorizer().fit(data)
    tdata = list(vect.transform(data).toarray())
    return tdata, vect

In [ ]:
def tokenize_lemmatize_text(data, debug_flag=0,rexp=r'[a-zA-Z][a-zA-Z]+'):
    data = data.str.lower()
    dtext=data.to_string(index=False)
    tonolizer = nltk.RegexpTokenizer(rexp)
    wtoken=tonolizer.tokenize(dtext)
    if debug_flag:
        dtoken = nltk.word_tokenize(dtext)
        print('default:',len(dtoken),len(set(dtoken)))
        tonolizer = nltk.RegexpTokenizer(r'\w+')
        dtoken=tonolizer.tokenize(dtext)
        print('word w+', len(dtoken), len(set(dtoken)))
        print('wordlen2+',len(wtoken), len(set(wtoken)))

    tags = nltk.pos_tag(wtoken)
    dtoken = [word for word, pos in tags if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS')]
    lemmatizer = WordNetLemmatizer()
    ltoken= [lemmatizer.lemmatize(word, wn.NOUN) for word, pos in tags if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS')]
    if debug_flag:
        print('Noun only', len(dtoken), len(set(dtoken)))
        print('lemmatizer:',len(ltoken), len(set(ltoken)))    
        ddict = nltk.FreqDist(dtoken)
        ddict = sorted(ddict.items(), key=lambda kv:[kv[1],kv[0]], reverse=True)
        print(ddict[0:10])
    
    return ltoken

In [ ]:
def ingredients_vectorizer(data, debug_flag=0):   
    ltoken = tokenize_lemmatize_text(data)
    mystop=['ml','kg','gram','tsp','tbsp','oz','pack','slices','self','lb','cut','tin','medium','small','large']

    vect = CountVectorizer(stop_words=mystop,min_df=5)
    vect.fit(ltoken)
    data_vect = vect.transform(data).toarray()
    
    data=data.squeeze()
    data = data.str.split(r'|', expand=True)
    ncols = data.shape[1]
    vct_num = []
    for i in range(data.shape[0]):
        nnn = ncols - data.iloc[i].isnull().sum()
        if nnn>30 and debug_flag:
            print(i, nnn)
        vct_num.append(nnn)

    return vct_num, data_vect, vect

In [ ]:
def title_vectorizer(data):
    ltoken = tokenize_lemmatize_text(data)
    vect = CountVectorizer(min_df=3)
    vect.fit(ltoken)
    data_vect = vect.transform(data).toarray()
    return data_vect, vect

def text_vectorizer(data, min_df=3):
    ltoken = tokenize_lemmatize_text(data,rexp=r'\w+')
    vect = TfidfVectorizer(min_df=min_df)
    vect.fit(ltoken)
    data_vect = vect.transform(data).toarray()
    return data_vect, vect

In [ ]:
def handel_all_none_values(data, dropna_flag=0):
    if dropna_flag:
        data = data.dropna(how='any',axis=0)
        return data
    num_null=[data[col].isnull().sum() for col in data.columns]
    print('Column Names:', data.columns)
    print('Numbers of NULLs/column', num_null)
    
    for col in ['total_time']:
        data[col] = data[col].fillna('PT0M')
    data['serving']         = data['serving'].fillna('0')
    data['nutritions_info'] = data['nutritions_info'].fillna('calories:0 calories')
    data['ingredients']     = data['ingredients'].fillna('noingredients')
    data['keywords']        = data['keywords'].fillna('nokeywords')
    data['total_ratings']   = data['total_ratings'].fillna('0 reviews')
    
    print('Number of NULLs at return = ', data.isnull().sum().sum())
    print('Data Sizes =', data.shape)
    return data

def prepare_and_clean_data_model(df):
    df = handel_all_none_values(df)

    #check for NULL raw data
    print('# of nan =', df.isnull().sum().sum())

    # change datatype 
    df['total_ratings']=df['total_ratings'].str.split().str[0].astype(int)
    df['total_time'] = time_to_minutes(df['total_time'])
    df['published_date'] = date_to_timestamp(df['published_date'])
    df['serving'] = serving_values(df['serving'])
    df['author'], author_le = author_encode(df['author'])
    data1 = nutrition_information_values(df['nutritions_info'])
    df =df.merge(data1,left_index=True,right_index=True)
    df.drop(columns=['nutritions_info'],inplace=True)

    # tokenization 
    df['keywords'], keywords_vect = keywords_vectorizer(df['keywords'])
    df['ingredients_counts'],df_ingre, ingre_vect = ingredients_vectorizer(df['ingredients'])
    df_title, title_vect = title_vectorizer(df['title'])
    df_descr, descr_vect = text_vectorizer(df['description'],min_df=7)
    
    df_text = [df_title, df_ingre, df_descr]
    vect_text = [title_vect, ingre_vect, descr_vect]
    return df, df_text, vect_text

#df = pd.read_csv('/content/drive/MyDrive/food_recipes_data.csv')
#df = pd.read_csv('/content/drive/MyDrive/865 Group/food_recipes_data.csv') #Penny

df, df_text, vect_text = prepare_and_clean_data_model(df)

print('df shape=',df.shape)
print('title:',df_text[0].shape)
print('ingredients:',df_text[1].shape)
print('description:',df_text[1].shape)

Column Names: Index(['total_time', 'cook_time', 'serving', 'crawled_at', 'description',
       'title', 'url', 'nutritions_info', 'image', 'ingredients', 'uniq_id',
       'source', 'author', 'prep_time', 'published_date', 'keywords',
       'total_ratings', 'instructions'],
      dtype='object')
Numbers of NULLs/column [2011, 3859, 2076, 0, 0, 0, 0, 2197, 0, 137, 0, 0, 0, 3090, 0, 360, 53, 1959]
Number of NULLs at return =  8908
Data Sizes = (14200, 18)
# of nan = 8908


NameError: ignored

In [ ]:
#delete column that we don't use
del df['url']
del df['crawled_at']
del df['prep_time']
del df['cook_time']
del df['uniq_id']
del df['source']

In [ ]:
del df['instructions']

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()